In [1]:
import jax
import jax.numpy as jnp 
import numpy as np
from transformers import Transformer

In [2]:
transformer = Transformer(seed=123, num_vocab=10, model_dim=4, hidden_dim=4, \
                          num_attention_layers=1, dk=2, dv=2)

In [3]:
input_sample = jnp.arange(6)
output_sample = jnp.arange(6)

In [4]:
transformer.forward(input_sample, output_sample)

Array([[ 1.6862992 ,  1.1083239 , -0.07306164,  0.8985744 ],
       [ 1.6494057 ,  1.0783671 , -0.07984804,  0.86864406],
       [ 1.6730537 ,  1.0975615 , -0.07548345,  0.88780427],
       [-1.0168165 , -0.8890565 , -0.90460634, -0.76273125],
       [-1.0168196 , -0.889058  , -0.904608  , -0.762732  ],
       [-1.0168185 , -0.8890561 , -0.9046065 , -0.76273143]],      dtype=float32)